<a href="https://colab.research.google.com/github/simonbustamante/mit-algoritmos-de-regresion/blob/master/Caso_de_Estudio_2_1_Predicci%C3%B3n_de_salarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 2.1 - Predicción de salarios

Configuración del *notebook*:

Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [13]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from rpy2.robjects import R

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#data_drop = drive.CreateFile({'id':'1LbJhqynkfcmezt1AFsHCJcTHdAsr1M2b'})
data_drop = drive.CreateFile({'id':'1U5UQdm43w9zh2w1VDxevQsb-vp3T7YCj'})
data_drop.GetContentFile('pay.discrimination.Rdata')

print('Google Drive sincronizado con éxito!')

Google Drive sincronizado con éxito!


In [14]:
! pip install rpy2==3.5.1
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [52]:
import rpy2.robjects as ro
import pandas as pd
from rpy2.robjects import pandas2ri

# Cargar el archivo .Rdata
ro.r['load']("pay.discrimination.Rdata")

# Activar la conversión automática de R a pandas DataFrame
pandas2ri.activate()

# Accede directamente al DataFrame de R convertido a un DataFrame de pandas
# Asegurándose de que la conversión automática está activada.
data = pandas2ri.conversion.rpy2py(ro.r['data'])

# Verificar que 'data' es un DataFrame de pandas
print(type(data))

# Asumiendo que 'data' es un DataFrame, mostrar las dimensiones
dims = data.shape
print(f"El DataFrame tiene {dims[0]} filas y {dims[1]} columnas.")

# Mostrar los primeros 10 registros del DataFrame
print("Primeros 10 registros del DataFrame:")
print(data.head(10))




<class 'pandas.core.frame.DataFrame'>
El DataFrame tiene 3835 filas y 12 columnas.
Primeros 10 registros del DataFrame:
         female   cg   sc  hsg   mw   so   we   ne  exp1     exp2       exp3  \
1663473     0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  33.0  10.8900  35.937000   
1663483     0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  27.0   7.2900  19.683000   
1663490     0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  13.0   1.6900   2.197000   
1663497     0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0   2.0   0.0400   0.008000   
1663500     1.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  15.0   2.2500   3.375000   
1663511     0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0   6.5   0.4225   0.274625   
1663512     1.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0   6.0   0.3600   0.216000   
1663543     0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  25.0   6.2500  15.625000   
1663557     0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  14.0   1.9600   2.744000   
1663592     0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  26.0   6.7600  17.576000   


In [53]:
%%R
load("pay.discrimination.Rdata")
dims  <- dim(data)
dims

[1] 3835   12


## Datos


In [54]:
%%R

# Cargar la base de datos
load("pay.discrimination.Rdata")
"pay.discrimination.Rdata"

# Mostrar las variables de la base de datos
#class(data)
str(data)

# Mostrar dimensiones de la base de datos
dims  <- dim(data)
cat('\nDimensiones de la base de datos:',toString(dims),'\n',fill = TRUE)
#print('\nDimensiones de la base de datos:',toString(dims),'\n',fill = TRUE)

'data.frame':	3835 obs. of  12 variables:
 $ female: num  0 0 0 0 1 0 1 0 0 0 ...
 $ cg    : num  0 1 0 1 1 0 0 0 0 0 ...
 $ sc    : num  0 0 1 0 0 1 1 0 0 1 ...
 $ hsg   : num  1 0 0 0 0 0 0 1 1 0 ...
 $ mw    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ so    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ we    : num  0 0 0 0 0 0 0 0 0 0 ...
 $ ne    : num  1 1 1 1 1 1 1 1 1 1 ...
 $ exp1  : num  33 27 13 2 15 6.5 6 25 14 26 ...
 $ exp2  : num  10.89 7.29 1.69 0.04 2.25 ...
 $ exp3  : num  35.937 19.683 2.197 0.008 3.375 ...
 $ wage  : num  11.66 12.82 5.78 12.47 18.52 ...

Dimensiones de la base de datos: 3835, 12 



Elaboración de una tabla con la media de cada variable:

In [28]:
%%R
# Tabla con la media de cada variable
stats <- as.matrix(apply(data, 2, mean))

# Asignar el nombre de la columna a stats
colnames(stats) = "KWEAAA"
print(stats,digits=2)

       KWEAAA
female   0.42
cg       0.38
sc       0.32
hsg      0.30
mw       0.29
so       0.24
we       0.21
ne       0.26
exp1    13.35
exp2     2.53
exp3     5.81
wage    15.53


## Metodología

### Sin separación de datos

Modelo básico:

In [29]:
%%R
# Regresión lineal del salario
fmla1     <-  wage ~ female + sc + hsg + cg + mw + so + we + ne + exp1 + exp2 + exp3

# Ejecución de la especificación lineal y cálculo del EMC y la R^2
full.fit1 <-  lm(fmla1, data=data)
fit1      <-  summary(full.fit1)
R2.1      <-  fit1$r.squared
R2.adj1   <-  fit1$adj.r.squared
n1        <-  length(fit1$res)
p1        <-  fit1$df[1]
MSE.adj1  <-  (n1/(n1-p1))*mean(fit1$res^2)

Modelo flexible:

In [30]:
%%R
# Regresión lineal: especificación cuadrática
# El operador ^ en el objeto formula especifica el orden de las interacciones
fmla2     <- wage ~  female + (sc + hsg + cg + mw + so + we + ne + exp1 + exp2 + exp3)^2

# Ejecución de la especificación cuadrática y cálculo del EMC y la R^2<- wage ~  female + (sc + hsg + cg + mw + so + we + ne + exp1 + exp2 + exp3)^
full.fit2 <- lm(fmla2, data=data)
fit2      <- summary(full.fit2)
R2.2      <- fit2$r.squared
R2.adj2   <- fit2$adj.r.squared
n2        <- length(fit2$res)
p2        <- fit2$df[1]
MSE.adj2  <- (n2/(n2-p2))*mean(fit2$res^2)

In [31]:
%%R
# Resumen de las especificaciones lineal y cuadrática
table1     <- matrix(0, 2, 4)
table1[1,] <- c(p1, R2.1, R2.adj1, MSE.adj1)
table1[2,] <- c(p2, R2.2, R2.adj2, MSE.adj2)

# Asignar nombres de filas y columnas
colnames(table1) <- c("p", "R^2 muestra", "R^2 ajustada", "EMC ajustado")
rownames(table1) <- c("reg basica", "reg flexible")


### Con separación de datos

Modelo básico:

In [33]:
%%R
# Estableciendo el generador de números aleatorios
set.seed(4)

# Regresión lineal del salario
fmla1     <-  wage ~ female + sc + hsg + cg + mw + so + we + ne + exp1 + exp2 + exp3

# Separación de los datos en los conjuntos de entrenamiento y prueba
train      <- sample(1:nrow(data), nrow(data)/2)

# Ejecución de la especificación lineal y cálculo del EMC y la R^2 con la muestra de prueba
full.fit1  <- lm(fmla1, data=data[train,])
yhat.fit1  <- predict(full.fit1, newdata=data[-train,])
y.test     <- data[-train,]$wage
MSE.fit1   <- summary(lm((y.test-yhat.fit1)^2~1))$coef[1]
R2.fit1    <- 1- MSE.fit1/var(y.test)

Modelo flexible:

In [34]:
%%R
# Separación de los datos en los conjuntos de entrenamiento y prueba
train      <- sample(1:nrow(data), nrow(data)/2)
# Regresión lineal: especificación cuadrática
# El operador ^ en el objeto formula especifica el orden de las interacciones
fmla2     <- wage ~  female + (sc + hsg + cg + mw + so + we + ne + exp1 + exp2 + exp3)^2
# Ejecución de la especificación cuadrática y cálculo del EMC y la R^2 con la muestra de prueba
full.fit2  <- lm(fmla2, data=data[train,])
yhat.fit2  <- predict(full.fit2, newdata=data[-train,])
y.test     <- data[-train,]$wage
MSE.fit2   <- summary(lm((y.test-yhat.fit2)^2~1))$coef[1]
R2.fit2    <- 1- MSE.fit2/var(y.test)

In [35]:
%%R
# Elaboración de la tabla de resultados
table2      <- matrix(0, 2, 3)
table2[1,]  <- c(p1, R2.fit1, MSE.fit1)
table2[2,]  <- c(p2, R2.fit2, MSE.fit2)

# Asignar nombres de filas y columnas
colnames(table2)  <- c("p ", "R^2 test", "EMC test")
rownames(table2)  <- c("reg basica", "reg flexible")

## Resultados

In [36]:
%%R
# Mostrando los resultados
cat('- Resultados sin separación de datos:\n',fill = TRUE)
print(table1,digits=4)

cat('\n\n- Resultados con separación de datos:\n',fill = TRUE)
print(table2,digits=4)

- Resultados sin separación de datos:

              p R^2 muestra R^2 ajustada EMC ajustado
reg basica   10     0.09549      0.09336        165.7
reg flexible 33     0.10397      0.09643        165.1


- Resultados con separación de datos:

             p  R^2 test EMC test
reg basica   10  0.12340   119.09
reg flexible 33  0.09647    99.86
